In [9]:
#!/usr/bin/python
# -*- coding: latin-1 -*-
"""This notebook reviews the contributions to WLF in Spain in YEAR"""

import os, sys, inspect

try :
    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[0:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

from io import StringIO
from datetime import datetime, timedelta
import pandas as pd
import re

In [10]:
YEAR                = 2016
TAG                 = 'WLF'
TAG_EXT             = 'Wiki Loves Folk'

BASE_WLF_NAME       = 'Commons:{1}/{0}'.format(YEAR, TAG_EXT)
BASE_WLF2016_NAME   = 'Commons:{1}/{0}'.format(2016, TAG_EXT)
LOG_PAGE            = BASE_WLF_NAME + "/Log"
FESTIVAL_DB_PAGE    = BASE_WLF2016_NAME + "/Festival DB"

WLF_CATEGORY          = "Category:Images from {1} {0} in Spain".format(YEAR, TAG_EXT)
NO_ID_CATEGORY        = "Category:Images from {1} {0} in Spain with no ID".format(YEAR, TAG)
WRONG_ID_CATEGORY     = "Category:Images from {1} {0} in Spain with no valid ID".format(YEAR, TAG)

commons_site = pb.Site("commons", "commons")

In [12]:
additional_festivals_df = pd.DataFrame(
    data=[
        {'name': 'Fallas de Denia', 
         'aut_com': 'Valencian Community', 
         'wikidata_id': 'Q47120057', 
         'category': 'Falles de Dénia'},
        {'name': 'Semana Santa de Aranjuez', 
         'aut_com': 'Community of Madrid', 
         'wikidata_id': 'Q47119796', 
         'category': 'Holy Week in Aranjuez'},
        {'name': 'Fiesta de la Virgen de Agosto y San Roque' , 
         'aut_com': 'Valencian Community', 
         'wikidata_id': u'Q47119479', 
         'category': 'Fiesta de la Virgen de Agosto y San Roque, Requena'},
        {'name': 'Gran Feria de Valencia', 
         'aut_com': 'Valencian Community', 
         'wikidata_id': 'Q12175110', 
         'category': 'Gran Fira de València'},
        {'name': 'Fiesta en honor de la Virgen de los Desamparados (Jérica)', 
         'aut_com': 'Valencian Community', 
         'wikidata_id': 'Q47128911', 
         'category': 'Fiesta en honor de la Virgen de los Desamparados, Jérica'},
        {'name': 'Fallas de Alcácer', 
         'aut_com': 'Valencian Community', 
         'wikidata_id': 'Q47129466', 
         'category': "Falles d'Alcàsser"},
        {'name': 'Semana Santa de Candelario', 
         'aut_com': 'Castile and León', 
         'wikidata_id': 'Q47128318', 
         'category': 'Holy Week in Candelario'},
        {'name': 'Festividad de San Vicente Ferrer', 
         'aut_com': 'Valencian Community', 
         'wikidata_id': 'Q5861379', 
         'category': 'Festivals of Saint Vincent Ferrer in the city of Valencia'}
    ]
)
additional_festivals = additional_festivals_df['wikidata_id'].values
additional_festivals = []
additional_festivals

[]

In [13]:
cat_wlf = pb.Category(commons_site, WLF_CATEGORY)
gen_wlf = pagegenerators.CategorizedPageGenerator(cat_wlf)

images_wlf = [page.title(withNamespace=True) for page in gen_wlf if page.is_filepage()]
len(images_wlf)

1621

In [14]:
pb.output('Retrieving --> {0} in Spain Festivals list from cache'.format(TAG))
festival_list_page = pb.Page(commons_site, FESTIVAL_DB_PAGE)
festival_list_text = StringIO(festival_list_page.text[festival_list_page.text.find('\n') + 1:festival_list_page.text.rfind('\n')])
festivals_df = pd.read_csv(festival_list_text, sep=";", index_col=False, names=['name', 'aut_com', 'wikidata_id', 'wikidata_timestamp', 'category', 'cat_timestamp', 'image'])

festivals = festivals_df["wikidata_id"].values

Retrieving --> WLF in Spain Festivals list from cache


In [15]:
for image_counter, image in enumerate(images_wlf):
    page = pb.Page(commons_site, image)
    text = page.text
    wikicode = mwh.parse(text)
    templates = wikicode.filter_templates()
    
    if (image_counter != 0) and (image_counter % 50 == 0) :
        pb.output ('Reviewing --> %d image pages downloaded' %(image_counter))

    cats = [cat.title(withNamespace=True) for cat in textlib.getCategoryLinks(text)]
    
    WLF_template_found = False
    WLF_identifier = u''
    wlf_templates = [template for template in wikicode.filter_templates() 
                    if template.name.lower().strip() == u"wlf"]
    
    new_cats = []

    if len(wlf_templates) > 0:
        WLF_template_found = True
        WLF_identifier = wlf_templates[0].get(1).value.strip()
        if (WLF_identifier not in festivals) and (WLF_identifier not in additional_festivals):
            print ('Festival identifier not valid')
            if WRONG_ID_CATEGORY not in cats :
                new_cats.append(WRONG_ID_CATEGORY)
            WLF_identifier = u''
            
    else:
        if NO_ID_CATEGORY not in cats :
            new_cats.append(NO_ID_CATEGORY)

    author_cat = u"Category:Images from {1} {0} in Spain by {2}".format(YEAR, TAG_EXT, page.oldest_revision["user"])
    author_cat_page = pb.Page(commons_site, author_cat)
    if not author_cat_page.exists():
        author_cat_page_text = u'{{{{hiddencat}}}}\n[[Category:Images from {1} {0} in Spain by author|{2}]]'.format(YEAR, TAG_EXT, page.oldest_revision["user"])
        author_cat_page.text = author_cat_page_text
        author_cat_page.save("{1} {0} in Spain: Authorshipt classification".format(YEAR, TAG))

    if author_cat not in cats:
        new_cats.append(author_cat)

    if (WLF_identifier != u'') and ((WLF_identifier in festivals) or (WLF_identifier in additional_festivals)) :
        festival_cat = u'Category:Images of festival with code {0} from {2} {1} in Spain'.format(WLF_identifier, YEAR, TAG_EXT)
        festival_cat_page = pb.Page(commons_site, festival_cat)
        if not festival_cat_page.exists():
            festival_cat_page_text = u'{{{{hiddencat}}}}\n[[Category:Images from {1} {0} in Spain by festival| {2}]]'.format(YEAR, TAG_EXT, WLF_identifier)
            festival_cat_page.text = festival_cat_page_text
            festival_cat_page.save("{1} {0} in Spain: Festival classification".format(YEAR, TAG))

        if festival_cat not in cats:
            new_cats.append(festival_cat)
            
    if len(new_cats) > 0 :
        cat_string = '\n'.join(['[[' + i + ']]' for i in new_cats])
        updated_text = text.replace(u'[[{0}]]\n'.format(NO_ID_CATEGORY), u'') + '\n' + cat_string
        page.text = updated_text
        page.save("{1} {0} in Spain: Classification".format(YEAR, TAG))

Page [[commons:Category:Images from Wiki Loves Folk 2016 in Spain by Tordo12]] saved
Sleeping for 19.5 seconds, 2018-01-14 01:02:10
Page [[commons:Category:Images of festival with code Q7884112 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 19.6 seconds, 2018-01-14 01:02:30
Page [[commons:File:Agrupacion de las Santas Mujeres.jpg]] saved
Sleeping for 19.2 seconds, 2018-01-14 01:02:50
Page [[commons:Category:Images from Wiki Loves Folk 2016 in Spain by Elnafe]] saved
Sleeping for 19.3 seconds, 2018-01-14 01:03:10
Page [[commons:Category:Images of festival with code Q28157451 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 19.6 seconds, 2018-01-14 01:03:30
Page [[commons:File:Al solecito.jpg]] saved
Sleeping for 19.1 seconds, 2018-01-14 01:03:50
Page [[commons:Category:Images from Wiki Loves Folk 2016 in Spain by Antonio Arenas]] saved
Sleeping for 19.5 seconds, 2018-01-14 01:04:10
Page [[commons:Category:Images of festival with code Q23459821 from Wiki Loves Folk 2016 i

Page [[commons:File:Archicofradía de Nuestro Padre Jesús Nazareno, Aranjuez 02.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:19:21
Page [[commons:File:Archicofradía de Nuestro Padre Jesús Nazareno, Aranjuez 03.jpg]] saved
Sleeping for 9.4 seconds, 2018-01-14 01:19:31
Page [[commons:File:Archicofradía de Nuestro Padre Jesús Nazareno, Aranjuez 04.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:19:41
Page [[commons:File:Archicofradía de Nuestro Padre Jesús Nazareno, Aranjuez 05.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:19:51
Page [[commons:File:Archicofradía de Nuestro Padre Jesús Nazareno, Aranjuez 06.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:20:01
Page [[commons:File:Archicofradía de Nuestro Padre Jesús Nazareno, Aranjuez 07.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:20:11
Page [[commons:File:Archicofradía de Nuestro Padre Jesús Nazareno, Aranjuez 08.jpg]] saved
Sleeping for 9.4 seconds, 2018-01-14 01:20:21
Page [[commons:File:Archicofradía de Nues

Page [[commons:Category:Images from Wiki Loves Folk 2016 in Spain by Niv66]] saved
Sleeping for 9.5 seconds, 2018-01-14 01:29:31
Page [[commons:Category:Images of festival with code Q23452764 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 9.6 seconds, 2018-01-14 01:29:41
Page [[commons:File:Calle Real después del encierro.jpg]] saved
Sleeping for 9.1 seconds, 2018-01-14 01:29:52
Page [[commons:File:Calles iluminadas.jpg]] saved
Sleeping for 9.1 seconds, 2018-01-14 01:30:01
Page [[commons:Category:Images of festival with code Q17636451 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 9.6 seconds, 2018-01-14 01:30:11
Page [[commons:File:Cantaderas2004.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:30:21
Page [[commons:File:Cantante Cuesta de Gos.jpg]] saved
Sleeping for 9.1 seconds, 2018-01-14 01:30:31
Page [[commons:Category:Images from Wiki Loves Folk 2016 in Spain by Majopegolina]] saved
Sleeping for 9.5 seconds, 2018-01-14 01:30:41
Page [[commons:Category:Images 

Page [[commons:File:Cautivo - Nazareno.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:41:12
Page [[commons:File:Cautivo - Nazarenos.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:41:22
Page [[commons:File:Cautivo - Regulares.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:41:32
Page [[commons:File:Cautivo - Trono Cautivo.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:41:42
Page [[commons:File:Cautivo - Trono de la Virgen.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:41:52
Page [[commons:File:Cautivo - Trono del Cautivo 2.jpg]] saved
Sleeping for 9.1 seconds, 2018-01-14 01:42:02
Page [[commons:Category:Images of festival with code Q23663100 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 9.5 seconds, 2018-01-14 01:42:12
Page [[commons:File:Cavalcade of Magi of Vila-real 2007.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:42:22
Page [[commons:File:Cavalcade of Magi of Vila-real 2013.JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:42:32
Page [[com

Sleeping for 9.3 seconds, 2018-01-14 01:53:13
Page [[commons:File:Consuelo Jerez2016 P1010785.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:53:23
Page [[commons:File:Consuelo Jerez2016 P1010786.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:53:33
Page [[commons:File:Consuelo Jerez2016 P1010787.JPG]] saved
Sleeping for 9.1 seconds, 2018-01-14 01:53:43
Page [[commons:File:Consuelo Jerez2016 P1010788.JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:53:53
Page [[commons:File:Consuelo Jerez2016 P1010789.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:54:03
Page [[commons:File:Consuelo Jerez2016 P1010790.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 01:54:13
Page [[commons:File:Consuelo Jerez2016 P1010791.JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:54:23
Page [[commons:File:Consuelo Jerez2016 P1010792.JPG]] saved
Sleeping for 9.0 seconds, 2018-01-14 01:54:33
Page [[commons:File:Consuelo Jerez2016 P1010793.JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 01:54:43


...

Sleeping for 9.3 seconds, 2018-01-14 05:11:23
Page [[commons:File:Sant Cristòfol de ses Corregudes - 38 (Es Migjorn Gran, 2 de agosto de 2015).JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:11:33
Page [[commons:File:Sant Cristòfol de ses Corregudes - 39 (Es Migjorn Gran, 2 de agosto de 2015).JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:11:43
Page [[commons:File:Sant Cristòfol de ses Corregudes - 40 (Es Migjorn Gran, 2 de agosto de 2015).JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:11:53
Page [[commons:File:Sant Cristòfol de ses Corregudes - 41 (Es Migjorn Gran, 2 de agosto de 2015).JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:12:03
Page [[commons:File:Sant Cristòfol de ses Corregudes - 42 (Es Migjorn Gran, 2 de agosto de 2015).JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:12:13
Page [[commons:File:Sant Cristòfol de ses Corregudes - 43 (Es Migjorn Gran, 2 de agosto de 2015).JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:12:23
Page [[commons:File:Sant

Page [[commons:Category:Images of festival with code Q9075874 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 9.6 seconds, 2018-01-14 05:22:33
Page [[commons:File:SemanaSantaLeon2014 (1).jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:22:44
Page [[commons:File:SemanaSantaLeon2014 (2).jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:22:54
Page [[commons:File:SemanaSantaLeon2014 (3).jpg]] saved
Reviewing --> 1400 image pages downloaded
Sleeping for 9.3 seconds, 2018-01-14 05:23:04
Page [[commons:File:SemanaSantaLeon2014 (4).jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:23:14
Page [[commons:File:SemanaSantaLeon2014 (5).jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:23:24
Page [[commons:File:Sentenca - Mayordomo.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:23:34
Page [[commons:File:Sentencia - Bocina.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:23:44
Page [[commons:File:Sentencia - Bola.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:23:54
Page [

Sleeping for 9.3 seconds, 2018-01-14 05:34:34
Page [[commons:File:Traje de Jarramplas (Piornal, Cáceres) en el Museo de Artes y Tradiciones Populares (15 de julio de 2016, Madrid) 03.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:34:44
Page [[commons:Category:Images of festival with code Q5836853 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 9.6 seconds, 2018-01-14 05:34:54
Page [[commons:File:Traje de maya (Colmenar Viejo, Madrid) en el Museo de Artes y Tradiciones Populares (15 de julio de 2016, Madrid).jpg]] saved
Sleeping for 9.1 seconds, 2018-01-14 05:35:05
Page [[commons:Category:Images of festival with code Q5752861 from Wiki Loves Folk 2016 in Spain]] saved
Sleeping for 9.6 seconds, 2018-01-14 05:35:14
Page [[commons:File:Traje de Peliqueiro (Laza, Ourense) en el Museo de Artes y Tradiciones Populares (15 de julio de 2016, Madrid).jpg]] saved
Sleeping for 9.4 seconds, 2018-01-14 05:35:24
Page [[commons:File:Trajes de Encamisado y Lacayo de la Encamisada (Menasalb

Sleeping for 9.3 seconds, 2018-01-14 05:45:45
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4262.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:45:55
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4263.JPG]] saved
Sleeping for 9.1 seconds, 2018-01-14 05:46:05
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4264.JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:46:15
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4309.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:46:25
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4310.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:46:35
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4312.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:46:45
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4314.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:46:55
Page [[commons:File:ViernesSantoJerez2016-HdadExpiracion IMG 4315.JPG]] saved
Sleeping

Sleeping for 9.3 seconds, 2018-01-14 05:56:56
Page [[commons:File:ViernesSantoJerez2016-HdadSoledad IMG 4386.JPG]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:57:05
Page [[commons:File:ViernesSantoJerez2016-HdadSoledad IMG 4387.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:57:16
Page [[commons:File:ViernesSantoJerez2016-HdadSoledad IMG 4388.JPG]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:57:26
Page [[commons:File:Virgen de la Caridad 1.jpg]] saved
Reviewing --> 1600 image pages downloaded
Sleeping for 9.2 seconds, 2018-01-14 05:57:36
Page [[commons:File:Virgen de la Caridad Cartagena.jpg]] saved
Sleeping for 9.2 seconds, 2018-01-14 05:57:46
Page [[commons:File:Viñeros Campana 2.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:57:56
Page [[commons:File:Viñeros Campana.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:58:06
Page [[commons:File:Viñeros Cetro.jpg]] saved
Sleeping for 9.3 seconds, 2018-01-14 05:58:16
Page [[commons:File:Viñeros Cruz Guia.jpg]] saved
Sleepin